<a href="https://colab.research.google.com/github/prithwis/parashar21/blob/main/P21_MultiCast_v4_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](http://1.bp.blogspot.com/_5QlanosqxiQ/SXLOLTu7S7I/AAAAAAAAAm8/0r6C7lCxsic/S1600-R/p21-header-003.png) <br>


---


![CC-BY-SA](https://licensebuttons.net/l/by-sa/3.0/88x31.png) ||
[Prithwis Mukerjee](http://www.yantrajaal.com) ||
Website - [Parashar21](https://parashar21.blogspot.com)<br>

#Create Charts and store in MongoDB

#Install Swiss Ephemeris <br>

https://pypi.org/project/pyswisseph/  - Python Package Documentation <br>
https://www.astro.com/swisseph/swephinfo_e.htm <br>
https://www.astro.com/swisseph/swephprg.htm - Programmer Documentation <br>
https://pythonhosted.org/pyswisseph/swisseph-module.html

In [1]:
#!pip install simple-crypt
!pip install python_string_utils
!pip install pyswisseph
#!pip install pyswisseph==2.10.2.0.dev1
!apt install mongodb > /dev/null

     |████████████████████████████████| 521 kB 13.5 MB/s 
  Created wheel for pyswisseph: filename=pyswisseph-2.8.0.post1-cp37-cp37m-linux_x86_64.whl size=724382 sha256=d5b8552bc0a659ad1958825e09d685f9cc4d6f94169b90ec1810b199368947b1
  Stored in directory: /root/.cache/pip/wheels/32/81/77/a3f7c72bd124fbdfec5f468ce0669ac09ea98a9af2b99edd61
Successfully built pyswisseph




In [2]:
!wget -q http://www.astro.com/ftp/swisseph/ephe/seas_18.se1
!wget -q http://www.astro.com/ftp/swisseph/ephe/semo_18.se1
!wget -q http://www.astro.com/ftp/swisseph/ephe/sepl_18.se1
!mkdir ephe
!mv *.se1 ephe

In [3]:
import swisseph as swe
swe.version
#Constants and Flags necessary for this application are set here
#
swe.set_ephe_path('/content/ephe') # set path to ephemeris files
# Calendar : Julian or Gregorian
SE_GREG_CAL = 1
gregflag = SE_GREG_CAL
#
# Ayanamsha type : Lahiri = 1
swe.set_sid_mode(1)  # Lahiri Aynamsha
#
# whether speed will be calculated along with position of planet
SEFLG_SPEED = int(256)
iflag = SEFLG_SPEED
#hsys = P, Placidus
#ascii P = 080
hsysP =  bytes('P', 'utf-8')

In [4]:
# Subtracts angles (eg Ayanamsha) from Longitudes 
# Required for conversion from Tropical (Western) to Sidereal (Indian) zodiac
def p21Sub (x,y):
    retVal = round(x - y,3)
    if retVal < 0:
        return retVal + 360
    else:
        return retVal

#Load Data


In [6]:
import pandas as pd

In [7]:
!gdown -O peopleData.csv https://drive.google.com/uc?id=1NpWf1RCyDkXzr1x_AnK0RkXuale26nQ0 # 4 Person Test Data
#!sed -i '1d' peopleData.csv 
#!cat peopleData.csv
df = pd.read_csv('peopleData.csv')
df.head()

Downloading...
From: https://drive.google.com/uc?id=1NpWf1RCyDkXzr1x_AnK0RkXuale26nQ0
To: /content/peopleData.csv
100% 564/564 [00:00<00:00, 654kB/s]


,Gender,Voc1_Cat,Voc1_Det,Voc2_Cat,Voc2_Det,Voc3_Cat,Voc3_Det,DoB_Day,DoB_Mon,DoB_Year,DoB_Time,TZ_Off,TZ_OffHours,PoB_Lat,PoB_Lon,TZ_Name,TZ_Type,Name
0,M,Engineer,nil,Teacher,nil,PostGraduate,nil,15,10,1961,23:42,h5e30,5.5,22.57,88.37,IST,standard,somu
1,F,Author,nil,Homemaker,nil,PostGraduate,nil,19,12,1965,10:26,h5e30,5.5,22.57,88.37,IST,standard,gopa
2,M,Engineer,nil,Teacher,nil,PostGraduate,nil,9,10,1990,23:09,h5e30,5.5,22.57,88.37,IST,standard,rahool
3,F,Biologist,nil,Scientist,nil,PostGraduate,nil,12,9,1990,0:52,h5e30,5.5,22.87,88.37,IST,standard,nilanjan


#Required Functions

In [9]:
def parsePersonData(P):
    global chart
    global LBDY, LBDM,LBDD,LBTh,LBTm,LBTs,tZone,natalLAT,natalLON
    
    LBDY = P['DoB_Year']     # Year of Birth, Local Time
    LBDM = P['DoB_Mon']      # Month of Birth, Local Time
    LBDD = P['DoB_Day']      # Day of Birth, Local Time

    LBTh = int(P['DoB_Time'].split(":")[0])                           # Hour of Birth, Local Time, 24 Hour Clock
    LBTm = int(P['DoB_Time'].split(":")[1])                          # Minute of Birth, Local Time
    LBTs = 0                           # Second of Birth, Local Time

    tZone = P['TZ_OffHours']                         # Time Zone Offset. East of Greenwich is +ve
    #print('ppd', LBDY, LBDM,LBDD,LBTh,LBTm,LBTs,tZone)
    #print(P['Gender'])
    natalLAT = P['PoB_Lat']
    natalLON = P['PoB_Lon']
    ck = str(P['DoB_Year'])+str(P['DoB_Mon'])+str(P['DoB_Day'])+P['DoB_Time']+str(P['TZ_OffHours'])+str(P['PoB_Lat'])+str(P['PoB_Lon'])
    chart = {
        "pid":{
            "tags" : [P['Voc1_Cat'],P['Voc2_Cat'],P['Voc3_Cat']],
            "ck" : ck,
            "name": string_utils.shuffle(P['Name'])
        }
    }

In [10]:
def convertDates():
    global natalUT, ayanamsha
    # Date & Time of Birth
    #Time Zone Conversion
    UTCdt = swe.utc_time_zone(LBDY,LBDM,LBDD,LBTh,LBTm,LBTs,tZone)
    #print(UTCdt)
    # Date & Time of Birth
    # Convert UTC date to Julian Date
    JD = swe.utc_to_jd(UTCdt[0],UTCdt[1],UTCdt[2],UTCdt[3],UTCdt[4],0,gregflag) # Second Value = 0
    #print (JD[0], JD[1])
    # JD[0] is Ephemeris Time
    # JD[1] is Universal Time 
    natalUT = JD[1]
    #print(natalUT, 'Julian Date of DTofB in Universal Time')
    #Ayanamsha
    ayanamsha = swe.get_ayanamsa(natalUT)
    #print('Lahiri Ayanamsha :', ayanamsha)


In [11]:
def calculateGrahaPositions():
    global bLon,bRet,P
    #Position of Planets
    #
    # body 0 = Sun, 1 = Moon, 2 = Merc, 3 = Ven, 4 = Mars, 5 = Jup, 6 = Sat, 11 = True Node (Rahu)
    bLon = []
    bRet = []
    for body in [0,1,2,3,4,5,6,11]:
        pData = swe.calc_ut(natalUT,body,iflag)
        bLon.append(pData[0][0])
        if pData[0][3] >= 0:
            bRet.append(False)
        else :
            bRet.append(True)
    #for ix in range(len(bLon)) : print(ix, bLon[ix], bRet[ix])
    #-------------------------------------------------------------------
    #House Position and Ascendants
    P = swe.houses(natalUT,natalLAT,natalLON,hsysP)
    #P = swe.houses_ex(natalUT,natalLAT,natalLON,hsysP)
    #for ix in range(len(P[0])) : print(ix, P[0][ix])
    #print('--')
    #for ix in range(len(P[1])) : print(ix, P[1][ix])

In [12]:
def buildGLonGRet():
    global GLon, GRet
    GLon = {
        'Glon':{
        'La':p21Sub(P[1][0],ayanamsha),
        'Su':p21Sub(bLon[0],ayanamsha),
        'Mo':p21Sub(bLon[1],ayanamsha),
        'Ma':p21Sub(bLon[4],ayanamsha),
        'Me':p21Sub(bLon[2],ayanamsha),
        'Ju':p21Sub(bLon[5],ayanamsha),
        'Ve':p21Sub(bLon[3],ayanamsha),
        'Sa':p21Sub(bLon[6],ayanamsha),
        'Ra':p21Sub(bLon[7],ayanamsha),
        'Ke':p21Sub(p21Sub(bLon[7],ayanamsha),180)
        }
    }

    GRet = {
        'GRet':{
        'La':False,
        'Su':False,
        'Mo':False,
        'Ma':bRet[4],
        'Me':bRet[2],
        'Ju':bRet[5],
        'Ve':bRet[3],
        'Sa':bRet[6],
        'Ra':False,
        'Ke':False
        }
    }


In [13]:
def appendDict(d1,d2):
    for i in range(len(d2)):
        k = list(d2.keys())[i]
        v = list(d2.values())[i]
        d1[k] = v

#Connect to MongoDB

In [14]:
from pymongo import MongoClient
import pandas as pd
import dateutil
import string_utils

In [15]:
from google.colab import drive
drive.mount('/content/drive')
# --------------------------------------------
# copy credentials file
#!ls /content/drive/'My Drive'/Praxis/WebCredentials
!cp /content/drive/'My Drive'/Praxis/WebCredentials/clevercloudMongoDB.py credentials.py
# -----------------------------------------------
# credential file for Prithwis Mukerjee
# this file needs to be uploaded into the VM

from credentials import rhostName,ruserName,rpassWord,rdbName,rMongoURI

#print(rhostName,ruserName,rdbName)
print(rdbName)
#print(rMongoURI)
# -----------------------------------------------
mongoURI = rMongoURI                                   # for remote server
#mongoURI = "mongodb://localhost:27017/"
client = MongoClient(mongoURI)
client.list_database_names() # ['admin', 'local']

Mounted at /content/drive
btncgfi3a5bxyea


['btncgfi3a5bxyea']

In [18]:
db = client.btncgfi3a5bxyea
# Show Collections. Query returns a Cursor [ ] ,  list it to see the content
list (db.list_collections())
# empty list '[]' means that there are not collections in database

[{'idIndex': {'key': {'_id': 1},
   'name': '_id_',
   'ns': 'btncgfi3a5bxyea.restaurants',
   'v': 2},
  'info': {'readOnly': False,
   'uuid': UUID('7554cbbb-c8f2-47a2-8429-426588529a48')},
  'name': 'restaurants',
  'options': {},
  'type': 'collection'}]

In [19]:
db = client.btncgfi3a5bxyea
db.p21Charts.drop()
db.p21Charts2.drop()
db.create_collection('p21Charts')       # Optional collection creation
kollection = db.p21Charts                   # Set the collection to work with

#Driver Code

In [20]:
for person in range(len(df)):
    personData =df.iloc[person]
    parsePersonData(personData)
    convertDates()
    calculateGrahaPositions()
    buildGLonGRet()
    #print(chart)
    #print(Glon)
    appendDict(chart,GLon)
    #print(chart)
    appendDict(chart,GRet)
    insert_result = kollection.insert_one(chart)
    insert_result.acknowledged    # Confirms that insert is successful
    #print(chart)

In [21]:
c = kollection.find()
for crnt in c:
    print(crnt)

{'_id': ObjectId('61dac319cef5589bbbc3b56f'), 'pid': {'tags': ['Engineer', 'Teacher', 'PostGraduate'], 'ck': '1961101523:425.522.5788.37', 'name': 'mous'}, 'Glon': {'La': 98.204, 'Su': 178.826, 'Mo': 250.52, 'Ma': 196.183, 'Me': 193.17, 'Ju': 274.781, 'Ve': 153.753, 'Sa': 270.179, 'Ra': 122.089, 'Ke': 302.089}, 'GRet': {'La': False, 'Su': False, 'Mo': False, 'Ma': False, 'Me': True, 'Ju': False, 'Ve': False, 'Sa': False, 'Ra': False, 'Ke': False}}
{'_id': ObjectId('61dac319cef5589bbbc3b570'), 'pid': {'tags': ['Author', 'Homemaker', 'PostGraduate'], 'ck': '1965121910:265.522.5788.37', 'name': 'gopa'}, 'Glon': {'La': 310.415, 'Su': 243.701, 'Mo': 201.001, 'Ma': 273.467, 'Me': 222.269, 'Ju': 62.748, 'Ve': 284.966, 'Sa': 318.17, 'Ra': 41.248, 'Ke': 221.248}, 'GRet': {'La': False, 'Su': False, 'Mo': False, 'Ma': False, 'Me': False, 'Ju': True, 'Ve': False, 'Sa': False, 'Ra': False, 'Ke': False}}
{'_id': ObjectId('61dac319cef5589bbbc3b571'), 'pid': {'tags': ['Engineer', 'Teacher', 'PostGradu

In [27]:
c = kollection.find({'pid.tags': {"$in": ['Scientist','Author']}})
for crnt in c:
    print(crnt)

{'_id': ObjectId('61dac319cef5589bbbc3b570'), 'pid': {'tags': ['Author', 'Homemaker', 'PostGraduate'], 'ck': '1965121910:265.522.5788.37', 'name': 'gopa'}, 'Glon': {'La': 310.415, 'Su': 243.701, 'Mo': 201.001, 'Ma': 273.467, 'Me': 222.269, 'Ju': 62.748, 'Ve': 284.966, 'Sa': 318.17, 'Ra': 41.248, 'Ke': 221.248}, 'GRet': {'La': False, 'Su': False, 'Mo': False, 'Ma': False, 'Me': False, 'Ju': True, 'Ve': False, 'Sa': False, 'Ra': False, 'Ke': False}}
{'_id': ObjectId('61dac319cef5589bbbc3b572'), 'pid': {'tags': ['Biologist', 'Scientist', 'PostGraduate'], 'ck': '19909120:525.522.8788.37', 'name': 'ijnnnlaa'}, 'Glon': {'La': 84.138, 'Su': 145.068, 'Mo': 54.233, 'Ma': 41.525, 'Me': 138.242, 'Ju': 101.216, 'Ve': 131.788, 'Sa': 265.084, 'Ra': 282.763, 'Ke': 102.763}, 'GRet': {'La': False, 'Su': False, 'Mo': False, 'Ma': False, 'Me': True, 'Ju': False, 'Ve': False, 'Sa': True, 'Ra': False, 'Ke': False}}


#Chronobooks <br>
![alt text](https://1.bp.blogspot.com/-lTiYBkU2qbU/X1er__fvnkI/AAAAAAAAjtE/GhDR3OEGJr4NG43fZPodrQD5kbxtnKebgCLcBGAsYHQ/s600/Footer2020-600x200.png)<hr>
Chronotantra and Chronoyantra are two science fiction novels that explore the collapse of human civilisation on Earth and then its rebirth and reincarnation both on Earth as well as on the distant worlds of Mars, Titan and Enceladus. But is it the human civilisation that is being reborn? Or is it some other sentience that is revealing itself. 
If you have an interest in AI and found this material useful, you may consider buying these novels, in paperback or kindle, from [http://bit.ly/chronobooks](http://bit.ly/chronobooks)